In [15]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [49]:

sz=224
batch_size=4

In [50]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input

In [51]:
train_data_dir = 'datanew/anime-cartoon/train'
validation_data_dir = 'datanew/anime-cartoon/valid'

In [67]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')
label_map = (train_generator.class_indices)
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

Found 208 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [53]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [54]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [55]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=2, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/2


/usr/lib/python3/dist-packages/PIL/Image.py:870: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


52/52 [==============================] - 68s 1s/step - loss: 2.1943 - acc: 0.7308 - val_loss: 1.4508 - val_acc: 0.7727
Epoch 2/2
52/52 [==============================] - 66s 1s/step - loss: 0.8991 - acc: 0.8558 - val_loss: 0.5487 - val_acc: 0.8864
CPU times: user 7min 27s, sys: 9.14 s, total: 7min 36s
Wall time: 2min 13s


In [56]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [59]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=2, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/2
11/52 [=====>........................] - ETA: 1:03 - loss: 0.2602 - acc: 0.8864

/usr/lib/python3/dist-packages/PIL/Image.py:870: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


52/52 [==============================] - 88s 2s/step - loss: 0.2443 - acc: 0.9279 - val_loss: 4.6662 - val_acc: 0.6364
Epoch 2/2
52/52 [==============================] - 85s 2s/step - loss: 0.1095 - acc: 0.9615 - val_loss: 0.7005 - val_acc: 0.8636
CPU times: user 9min 49s, sys: 17.1 s, total: 10min 6s
Wall time: 2min 52s


In [ ]:
model.save('my_model.h5')


In [68]:
label_map

{'anime': 0, 'cartoon': 1}